<a href="https://colab.research.google.com/github/fbeilstein/python/blob/master/lecture_04_modules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Modules

**Why use modules?**

*     Code reuse
*     System name-space partitioning
*     Implementing shared services or data

**Module topics**
 
*     The basics
*     Import variations
*     Reloading modules
*     Design concepts
*     Modules are objects
*     Package imports
*     Odds and ends
*     Module gotchas

###Module basics

*     Creating modules: Python files, C extensions; Java classes (Jython)
*     Using modules: import, from, reload(), 3.X: imp.reload()
*     Module search path: $PYTHONPATH

In [0]:
%%writefile module1.py

def printer(x): # module attribute
  print(x)

Writing module1.py


**Module usage**

In [0]:
import module1 # get module
module1.printer('Hello world!')

Hello world!


In [0]:
from module1 import printer # get an export
printer('Hello world!')

Hello world!


In [0]:
from module1 import * # get all exports
printer('Hello world!')

Hello world!


from * can obscure variables meaning

In [0]:
%%writefile module1.py

def func(): # module attribute
  print('module 1')

Overwriting module1.py


In [0]:
%%writefile module2.py

def func(): # module attribute
  print('module 2')

Writing module2.py


In [0]:
%%writefile module3.py

def func(): # module attribute
  print('module 3')

Writing module3.py


In [0]:
from module1 import * # may overwrite my names
from module2 import * # no way to tell what we get
from module3 import *

func() # ← ?!

# Advice: use from * with at most 1 module per file

module 3


from does not play well with reload

In [0]:
%%writefile moduleA.py

def func(): # module attribute
  print('module before change')

Writing moduleA.py


In [0]:
from moduleA import func # copy variable out
func() # test it

module before change


In [0]:
%%writefile moduleA.py
# change moduleA.py

def func(): # module attribute
  print('module AFTER change')

Overwriting moduleA.py


In [0]:
from imp import reload # required in 3.X
print("ERROR EXPECTED")
reload(moduleA) # <- FAILS: unbound name!

ERROR EXPECTED


NameError: ignored

In [0]:
import moduleA # must bind name here
reload(moduleA) # ok: loads new code
func() # <- FAILS: old object!

module before change


In [0]:
moduleA.func() # this works now

module AFTER change


In [0]:
from moduleA import func # so does this
func()

# Advice: don't do that--run scripts other ways

module AFTER change


###Module files are a namespace

*     A single scope: local==global
*     Module statements run on first import
*     Top-level assignments create module attributes
*     Module namespace: attribute __dict__, or dir()

In [0]:
%%writefile moduleB.py

print('starting to load')
import sys
name = 42
def func(): pass
class klass: pass
print('done loading.')

Writing moduleB.py


Usage

In [0]:
import moduleB

starting to load
done loading.


In [0]:
moduleB.sys

<module 'sys' (built-in)>

In [0]:
moduleB.name

42

In [0]:
moduleB.func, moduleB.klass

(<function moduleB.func>, moduleB.klass)

In [0]:
moduleB.__dict__.keys() # add list() in 3.X

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__file__', '__cached__', '__builtins__', 'sys', 'name', 'func', 'klass'])

###Name qualification

*     Simple variables
      -      X searches for name X in current scopes
*     Qualification
      -      X.Y searches for attribute Y in object X
*     Paths
      -      X.Y.Z gives a path of objects to be searched
*     Generality
      -      Qualification works on all objects with attributes: modules, classes, built-in types, etc.

![alt text](https://learning-python.com/class/Workbook/unit06_files/image002.gif)

###Import variants

*     Module import model
      -      module loaded and run on first import or from
      -      running a modules code creates its top-level names
      -      later import/from fetches already-loaded module
*     import and from are assignments
      -      import assigns an entire module object to a name
      -      from assigns selected module attributes to names

Operation |	 Interpretation
---|---
import mod |	fetch a module as a whole
from mod import name |	fetch a specific name from a module
from mod import * |	fetch all top-level names from a module
imp.reload(mod) |	force a reload of modules code

![alt text](https://learning-python.com/class/Workbook/unit06_files/image004.gif)

###Reloading modules

*     Imports only load/run module code first time
*     Later imports use already-loaded module
*     reload function forces module code reload/rerun
*     Allows programs to be changed without stopping
*     3.X: must first from imp import reload to use!

**General form**

```python
import module # initial import
[use module.attributes]
 # change module file

from imp import reload # required in 3.X
reload(module) # get updated exports
[use module.attributes]
```

**Usage details**

*     A function, not a statement
*     Requires a module object, not a name
*     Changes a module object in-place:
      -      runs module files new code in current namespace
      -      assignments replace top-level names with new values
      -      impacts all clients that use import to fetch module
      -      impacts future from clients (see earlier example)

**Reload example**

*     Changes and reload file without stopping Python
*     Other common uses: GUI callbacks, embedded code, etc.

In [0]:
%%writefile changer.py

message = 'First version'
def printer():
  print(message)

Overwriting changer.py


In [0]:
import changer
changer.printer()

First version


In [0]:
%%writefile changer.py

message = 'After editing'
def printer():
  print(message)

Overwriting changer.py


In [0]:
import changer
changer.printer() # no effect: uses loaded module

After editing


In [0]:
import importlib
importlib.reload(changer) # forces new code to load/run
changer.printer()

After editing


###Package imports


Module package imports name directory paths:
*      Module name -> dir.dir.dir in import statements and reloads
       -      import dir1.dir2.mod -> loads dir1\dir2\mod.py
       -      from dir1.dir2.mod import name
*      dir1 must be contained by a directory on sys.path (., PYTHONPATH, etc.)
*      Each dir must have \_\_init\_\_.py file, possibly empty (till 3.3: optional)
*      \_\_init\_\_.py gives directory's namespace, can use \_\_all\_\_ for from*
*      Simplifies path, disambiguates same-named modules files

**Example**

```
For:
    dir0\dir1\dir2\mod.py
And:
    import dir1.dir2.mod
``` 

-      dir0 (container) must be listed on the module search path
-      dir1 and dir2 both must contain an \_\_init\_\_.py file
-      dir0 does not require an \_\_init\_\_.py
 
```
dir0\
    dir1\
      __init__.py
      dir2\
          __init__.py
          mod.py
```

**Why packages?**
 

```
root\
    sys1\
        __init__.py (__init__ needed if dir in import)
        util.py
        main.py (import util finds here)
        other.py
    sys2\
        __init__.py
        util.py
        main.py
        other.py
    sys3\ (here or elsewhere)
        __init__.py (your new code here)
        myfile.py (import util depends on path)
                  (import sys1.util doesnt)
```

**Advanced: Relative import syntax (2.5+)**

To enable in 2.X (standard in 3.X):
```python
from __future__ import absolute_import # till 2.7, from stmt only
``` 

In code located in package folder pkg:

```python
import string # skips pkg: finds the standard library's version

from .string import name1, name2 # import names from pkg.string only
from . import string # import pkg.string
```

**Advanced: Namespace packages (3.3+)**

Extension to usual import algorithm: directories without \_\_init\_\_.py, located anywhere on path, checked for last, and used only if no normal module or package found at level: concatenation of all found becomes a virtual package for deeper imports. Not yet used much in practice.

###Odds and ends

*     Python 2.0+: import module as name
      -      Like import module + name = module
      -      Also good for packages: import sys1.util as util

*     Loading modules by name string
      -      exec(import + name)
      -      \_\_import\_\_(name)

*     Modules are compiled to byte code on first import
      -      .pyc files serve as recompile dependency
      -      compilation is automatic and hidden

*     Data hiding is a convention
      -      Exports all names defined at the top-level of a module
      -      Special case: \_\_all\_\_ list gives names exported by from *
      -      Special case: _X names aren't imported by a from*

*     The \_\_name\_\_ == \_\_main\_\_ trick
      -      \_\_name\_\_ auto set to \_\_main\_\_ only when run as script
      -      allows modules to be imported and/or run
      -      Simplest unit test protocol, dual usage modes for code

In [0]:
%%writefile tester.py

def func(s):
  print(s)

if __name__ == '__main__': # only when run
  func('This is Sparta') # not when imported

Writing tester.py


**Usage modes**

In [0]:
import tester
tester.func("This is Kyiv")

This is Kyiv


In [0]:
! python -u tester.py

This is Sparta


###Module design concepts

*     Always in a module: interactive = module \_\_main\_\_
*     Minimize module coupling: global variables
*     Maximize module cohesion: unified purpose
*     Modules should rarely change other modules variables

**Suppose mod.py contains**

```python
X = 99 # reader sees only this
```

**always okay to use**

```python
import mod
print(mod.X ** 2)
```

**almost always a Bad Idea!**

```python
import mod
mod.X = 88
```

 **better: isolates coupling**

```python
import mod
result = mod.func(88)
```

![alt text](https://learning-python.com/class/Workbook/unit06_files/image006.gif)

###Modules are objects: metaprograms

*     A module which lists namespaces of other modules
*     Special attributes: module.\_\_name\_\_, \_\_file\_\_, \_\_dict\_\_
*     getattr(object, name) fetches attributes by string name
*     Add to $PYTHONSTARTUP to preload automatically

**Example**

```python
verbose = 1

def listing(module):
  if verbose:
    print("-" * 30)
    print("name: %s file: %s" % (module.__name__, module.__file__))
    print("-" * 30)
 

  count = 0
  for attr in module.__dict__.keys(): # scan names
    print("%02d) %s" % (count, attr), end=",")
    if attr[0:2] == "__":
      print("<built-in name>") # skip specials
    else:
      print(getattr(module, attr)) #__dict__[attr]
    count = count+1
 

  if verbose:
    print("-" * 30)
    print(module.__name__, "has %d names" % count)
    print("-" * 30)

if __name__ == "__main__":
  import mydir
  listing(mydir) # self-test code: list myself
  ```

In [0]:
! curl -O https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_01_and_lecture_02_intro_and_python/mydir.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   621  100   621    0     0   2083      0 --:--:-- --:--:-- --:--:--  2083


*     Running the module on itself

In [0]:
! python -u mydir.py

------------------------------
name: mydir file: /content/mydir.py
------------------------------
00) __name__,<built-in name>
01) __doc__,<built-in name>
02) __package__,<built-in name>
03) __loader__,<built-in name>
04) __spec__,<built-in name>
05) __file__,<built-in name>
06) __cached__,<built-in name>
07) __builtins__,<built-in name>
08) verbose,1
09) listing,<function listing at 0x7f2fb5db5d90>
------------------------------
mydir has 10 names
------------------------------


**Another program about programs**

*     exec runs strings of Python code
*     os.system runs a system shell command
*     \_\_dict\_\_ attribute is module namespace dictionary
*     sys.modules is the loaded-module dictionary

In [0]:
def python(cmd):
  import importlib
  import __main__
  namespace = __main__.__dict__
  exec(cmd, namespace, namespace)

def fix(modname):
  import sys # edit,(re)load
  if modname in sys.modules.keys():
    python('import importlib; importlib.reload(' + modname + ')')
  else:
    python('import ' + modname)   

In [0]:
%%writefile tester.py

def func():
  print('UNEDITED')

Overwriting tester.py


In [0]:
import tester
tester.func()

UNEDITED


In [0]:
%%writefile tester.py

def func():
  print('hello world')

Overwriting tester.py


In [0]:
import tester
tester.func()

UNEDITED


In [0]:
fix("tester")
tester.func()

hello world


###Module gotchas

In [0]:
%%writefile nested1.py

X = 99
def printer(): print(X)

Writing nested1.py


In [0]:
%%writefile nested2.py

from nested1 import X, printer
X = 88
printer()

Writing nested2.py


In [0]:
%%writefile nested3.py

import nested1
nested1.X = 88
nested1.printer()

Overwriting nested3.py


In [0]:
! python -u nested2.py
! python -u nested3.py

99
88


Statement order matters at top-level
 
*     Solution: put most immediate code at bottom of file

In [0]:
#func1() # error: "func1" not yet assigned

def func1():
  print(func2()) # okay: "func2" looked up later

#func1() # error: "func2" not yet assihned

def func2():
  return "Hello"

func1() # okay: "func1" and "func2" assigned

Hello


Recursive from import gotchas

*     Solution: use import, or from inside functions

In [0]:
%%writefile recur1.py

X = 1
import recur2
Y = 2

Writing recur1.py


In [0]:
%%writefile recur2.py

from recur1 import X
from recur1 import Y

Writing recur2.py


In [0]:
print("ERROR EXPECTED")
import recur1

ERROR EXPECTED


ImportError: ignored

In [0]:
import recur2

In [0]:
from recur1 import Y # error: "Y" not yet assigned

**reload may not impact from imports**

*     reload overwrites existing module object
*     But from names have no link back to module
*     Use import to make reloads more effective

-> import module module.X reflects module reloads


-> from module import X X may not reflect module reloads!

**reload isn't applied transitively**

*     Use multiple reloads to update subcomponents
*     Or use recursion to traverse import dependencies

###Optional reading: a shared stack module

*     Manages a local stack, initialized on first import
*     All importers share the same stack: single instance
*     Stack accessed through exported functions
*     Stack can hold any kind of object (heterogeneous)

In [0]:
! curl -O https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_01_and_lecture_02_intro_and_python/mystack.py
! cat mystack.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   727  100   727    0     0   2506      0 --:--:-- --:--:-- --:--:--  2515
stack = [] # on first import
error = 'mystack.error' # local exceptions

def push(obj):
  global stack # 'global' to change
  stack = [obj] + stack # add item to front

def pop():
  global stack
  if not stack:
    raise Exception(error, 'stack underflow') # raise local error
  top, stack = stack[0], stack[1:] # remove front item
  return top

def top():
  if not stack: # raise local error
    raise Exception(error, 'stack underflow') # or let IndexError
  return stack[0]

def empty(): return not stack # is the stack []?
def member(obj): return obj in stack # item in stack?
def item(offset): return stack[offset] # index the stack
def length(): return len(stack) # number entries
def dump(): print('<Stack:%s>' % stack)


**Using the stack module**

In [0]:
import mystack
for i in range(5): mystack.push(i)

mystack.dump()

<Stack:[4, 3, 2, 1, 0]>


**Sequence-like tools**

In [0]:
mystack.item(0), mystack.item(-1), mystack.length()

(4, 0, 5)

In [0]:
mystack.pop(), mystack.top()

(4, 3)

In [0]:
mystack.member(4), mystack.member(3)

(False, True)

In [0]:
for i in range(mystack.length()): print(mystack.item(i),end=",")

3,2,1,0,

**Exceptions**

In [0]:
while not mystack.empty(): x = mystack.pop(),
try:
  mystack.pop()
except Exception as e:
  print(e.args)

('stack1.error', 'stack underflow')


**Module clients**

In [0]:
from mystack import *
push(123) # module-name not needed
dump()

<Stack:[123]>


In [0]:
import mystack
mystack.dump()
if not mystack.empty(): # qualify by module name
  x = mystack.pop()
mystack.push(1.23)
mystack.dump()

<Stack:[123]>
<Stack:[1.23]>
